In [1]:
import pandas as pd

In [16]:
df_retweets = pd.read_pickle("../data/voterfraud2020/df_retweets.pickle")
df_retweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25566698 entries, 0 to 25566697
Data columns (total 4 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   user                object
 1   timestamp           object
 2   retweeted           object
 3   retweetedFrom_user  object
dtypes: object(4)
memory usage: 780.2+ MB


In [ ]:
df_tweets = pd.read_pickle("../data/voterfraud2020/df_recent_tweets.pickle")
df_tweets.columns

In [4]:
df_tweets_slim = df_tweets[[
    'datastore_id',
    'urls',
    'hasMedia',
    'retweet_count',
    'hashtags',
    'quote_count', 
    'user', 
    'text', 
    'quote_tweet', 
    'timestamp'
]].copy()
df_tweets_slim["timestamp"] = pd.to_datetime(df_tweets_slim.timestamp)

In [5]:
from datetime import datetime
df_tweets_relevant = df_tweets_slim[
    (df_tweets_slim.timestamp.dt.tz_convert(None) <= pd.Timestamp(datetime(2020, 11, 21)))
    & (df_tweets_slim.timestamp.dt.tz_convert(None) >= pd.Timestamp(datetime(2020, 11, 1)))
]
df_tweets_relevant.shape

(3552105, 10)

In [18]:
from datetime import datetime
df_retweets["timestamp"] = pd.to_datetime(df_retweets["timestamp"])
df_retweets_relevant = df_retweets[
    (df_retweets.timestamp.dt.tz_convert(None) <= pd.Timestamp(datetime(2020, 11, 21)))
    & (df_retweets.timestamp.dt.tz_convert(None) >= pd.Timestamp(datetime(2020, 11, 1)))
]
df_retweets_relevant.shape

(10967079, 4)

In [ ]:
df_retweets_sample = df_retweets_relevant.sample(25000, random_state=42)
df_retweets_sample.to_pickle("../data/voterfraud2020/processed/df_retweets_sample.pickle")

In [6]:
df_tweets_relevant.to_pickle("../data/voterfraud2020/processed/df_tweets_relevant.pickle")

In [2]:
import pandas as pd
df_tweets_relevant = pd.read_pickle("../data/voterfraud2020/processed/df_tweets_relevant.pickle").rename(columns={
    "datastore_id": "tweet_id"
}).set_index("tweet_id")
df_tweets_relevant.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3552105 entries, 1327172028709801985 to 1324408015227932672
Data columns (total 9 columns):
 #   Column         Dtype              
---  ------         -----              
 0   urls           object             
 1   hasMedia       bool               
 2   retweet_count  int32              
 3   hashtags       object             
 4   quote_count    int32              
 5   user           object             
 6   text           object             
 7   quote_tweet    object             
 8   timestamp      datetime64[ns, UTC]
dtypes: bool(1), datetime64[ns, UTC](1), int32(2), object(5)
memory usage: 220.2+ MB


In [16]:
df_retweets_sample = pd.read_pickle("../data/voterfraud2020/processed/df_retweets_sample.pickle")
df_retweets_sample.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25000 entries, 12489425 to 24464681
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   user                25000 non-null  object             
 1   timestamp           25000 non-null  datetime64[ns, UTC]
 2   retweeted           25000 non-null  object             
 3   retweetedFrom_user  25000 non-null  object             
dtypes: datetime64[ns, UTC](1), object(3)
memory usage: 976.6+ KB


In [62]:
# Convert to correct format
import re
import string
normalize_spaces = lambda x: re.sub("[\n\r\t ]+", ' ', x)

df_retweets_sample["text"] = df_retweets_sample.retweeted.apply(
    lambda tweet_id: f"RT {normalize_spaces(df_tweets_relevant.loc[tweet_id].text)}" if tweet_id in df_tweets_relevant.index else None
)
df_retweets_sample["user_id"] = df_retweets_sample["user"]
df_retweets_sample["created_at"] = df_retweets_sample["timestamp"].apply(
    lambda t: str(t.timestamp() * 1000)
)
df_retweets_sample["quote_count"] = 0
df_retweets_sample["retweet_count"] = 0
df_retweets_sample["like_count"] = 0
df_retweets_sample["retweet_count"] = 0
df_retweets_sample["replied_to"] = None
df_retweets_sample["quoted"] = None
df_retweets_sample["mentions"] = df_retweets_sample["text"].apply(
    lambda text: [mention.strip(string.punctuation) for mention in text.split() if mention.startswith("@")] if text is not None else []
)
df_retweets_sample["hashtags"] = df_retweets_sample["text"].apply(
    lambda text: [tag.strip(string.punctuation) for tag in text.split() if tag.startswith("#")] if text is not None else []
)
df_retweets_sample["id"] = df_retweets_sample.apply(
    lambda t: f"{t.retweetedFrom_user}-{t.retweeted}",
    axis=1
)


df_retweets_processed = df_retweets_sample.sample(15000).drop(
    ["user", "retweetedFrom_user", "timestamp"], 
    axis=1
)[df_retweets_sample['text'].notna()].set_index("id")
df_retweets_processed

/var/folders/yb/l4pdzll51k946vtlt23kftkh0000gn/T/ipykernel_15764/1494163201.py:31: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_retweets_processed = df_retweets_sample.sample(15000).drop(


,retweeted,text,user_id,created,quote_count,retweet_count,like_count,replied_to,mentions,hashtags,created_at,quoted
id,,,,,,,,,,,,
24578794-1324110432811032585,1324110432811032585,RT Trump never expected to win in 2016. It was...,663573,1604540131000.0,0,0,0,None,[],[],1604540131000.0,None
17995040-1325554362031157250,1325554362031157250,RT The legal process must play out. The Trump ...,3533401217,1604911570000.0,0,0,0,None,[],[],1604911570000.0,None
2421067430-1326780407757623303,1326780407757623303,"RT When you find real voter fraud, why is alwa...",356229790,1605186746000.0,0,0,0,None,[],[],1605186746000.0,None
1035948712500056066-1325990247273033728,1325990247273033728,RT DOJ prosecutor overseeing voter fraud unit ...,81523937,1604981862000.0,0,0,0,None,[],[],1604981862000.0,None
78523300-1326048241385795584,1326048241385795584,RT Lindsey Graham: Possible ballot harvesting ...,1193708207052472320,1604992492000.0,0,0,0,None,[],[],1604992492000.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...
1239229116622266372-1327809106439237633,1327809106439237633,RT Executive Order on Imposing Certain Sanctio...,1160989742521098240,1605410866000.0,0,0,0,None,[],[],1605410866000.0,None
25073877-1327750127679889409,1327750127679889409,RT There is tremendous evidence of wide spread...,72939431,1605403312000.0,0,0,0,None,[],[],1605403312000.0,None
36715753-1328741298812182538,1328741298812182538,RT I'm not a patient person. I'm ready for vot...,2431360058,1605631949000.0,0,0,0,None,[],[],1605631949000.0,None


In [63]:
df_tweets_sample = df_tweets_relevant.sample(35000, random_state=42)
df_tweets_sample.head()

,urls,hasMedia,retweet_count,hashtags,quote_count,user,text,quote_tweet,timestamp
tweet_id,,,,,,,,,
1322913426495184896,[https://nypost.com/2020/09/29/minneapolis-pol...,False,0,[],0,1186512686,@FakeDanWeber https://t.co/5nL6rGlUOH,None,2020-11-01 14:48:34+00:00
1323843128432840705,[],False,0,[],0,41059238,There reports of hispanic interference in the ...,None,2020-11-04 04:22:52+00:00
1326071076024053761,[https://www.washingtonpost.com/nation/2020/11...,True,0,[],0,83879634,Pulitzer worthy.\n\nhttps://t.co/N63HwOkZwS ht...,None,2020-11-10 07:55:56+00:00
1323958392847818754,[https://twitter.com/i/web/status/132395839284...,False,0,"[Vote2020, USAElections2020, CorruptGOP, Trump...",0,1203925149625602048,@politico Our country is in grave danger. Elec...,None,2020-11-04 12:00:53+00:00
1324101615905853444,[http://www.donaldjtrump.com],True,0,[StopTheSteal],0,1270454004674412545,@ChatChristi THE FIGHT GOES ON\nDONATE TO htt...,None,2020-11-04 21:30:00+00:00


In [64]:
# add mentions, retweeted col
# fix user_id, id, in_reply_to_user_id cols

df_tweets_sample.index.rename("id", inplace=True)
df_tweets_sample["mentions"] = df_tweets_sample["text"].apply(
    lambda text: [mention.strip(string.punctuation) for mention in text.split() if mention.startswith("@")] if text is not None else []
)
df_tweets_sample["hashtags"] = df_tweets_sample["text"].apply(
    lambda text: [tag.strip(string.punctuation) for tag in text.split() if tag.startswith("#")] if text is not None else []
)

df_tweets_sample["user_id"] = df_tweets_sample["user"]
df_tweets_sample["retweeted"] = None
df_tweets_sample["quoted"] = df_tweets_sample["quote_tweet"]
df_tweets_sample["created_at"] = df_tweets_sample["timestamp"].apply(
    lambda t: str(t.timestamp() * 1000)
)

df_tweets_processed = df_tweets_sample.drop(
    ["user", "hasMedia", "timestamp", "urls", "quote_tweet"], 
    axis=1
)# [df_retweets_sample['text'].notna()].set_index("id")
df_tweets_processed.columns

Index(['retweet_count', 'hashtags', 'quote_count', 'text', 'mentions',
       'user_id', 'retweeted', 'quoted', 'created_at'],
      dtype='object')

In [65]:
final_cols = df_retweets_processed.columns.intersection(
    df_tweets_processed.columns
)
final_cols

Index(['retweeted', 'text', 'user_id', 'quote_count', 'retweet_count',
       'mentions', 'hashtags', 'created_at', 'quoted'],
      dtype='object')

In [68]:
df_combined = pd.concat([
    df_retweets_processed[final_cols].reset_index(),
    df_tweets_processed[final_cols].reset_index(),
]).sort_values("created_at", ascending=False)
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49878 entries, 33810 to 27288
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             49878 non-null  object
 1   retweeted      14878 non-null  object
 2   text           49878 non-null  object
 3   user_id        49878 non-null  object
 4   quote_count    49878 non-null  int64 
 5   retweet_count  49878 non-null  int64 
 6   mentions       49878 non-null  object
 7   hashtags       49878 non-null  object
 8   created_at     49878 non-null  object
 9   quoted         17237 non-null  object
dtypes: int64(2), object(8)
memory usage: 4.2+ MB


In [69]:
df_combined.to_json(
    "../data/voterfraud2020/processed/df_sample.json",
    orient="records"
)

```json
    {
        "id": "1354218419441315840",
        "text": "With today\u2019s winter weather\u2014and more in the forecast later this week\u2014resist the urge to hibernate and come explore everything the Park has to offer. Get an insider\u2019s look at the world\u2019s greatest urban park with the #CentralParkConservancy\u2019s Winter Guide. https:\/\/t.co\/OZJyg2EJGE https:\/\/t.co\/jL1QTFUTAf",
        "created_at": 1611705806000,
        "hashtags": [
            "centralparkconservancy"
        ],
        "mentions": [],
        "in_reply_to_user_id": null,
        "user_id": "96795310",
        "retweet_count": 0,
        "quote_count": 3,
        "reply_count": 3,
        "like_count": 109,
        "retweeted": null,
        "replied_to": null,
        "quoted": null,
        "is_anomaly": 0,
        "merlion_anomaly_total_count": 0.0,
        "merlion_anomaly_top1_hashtag_count": 1.0862331593,
        "merlion_anomaly_top2_hashtag_count": 0.6297762086,
        "merlion_anomaly_top3_hashtag_count": 0.0
    },
````